# 3. Fundamentals
This chapter covers the fundamental concepts needed to understand this thesis. Starting
from the joint assembly and its components, followed by Kinematic, Dynamic and Machine
Learning Concepts. This chapter aims to provide a brief summary and highlight specific
equations and key algorithmic implementations

## 3.1. Joint Assembly

<figure style="text-align: center;">
    <img src="./Resources/Axis3.png" id="Axis3" style="width:70%; margin: auto;">
    <figcaption align="center"> <b>Figure 3.1.: </b>Axis assembly of Joint-3 in LARA8, shown for reference purposes and not
as an exact depiction of the assembly. Adapted from <cite id="93kng"><a href="#zotero%7C17740253%2F4JU53NME">[1]</a></cite></figcaption>
</figure>

The general schema of the robot's axis is presented in <a href="#Axis3">Figure 3.1</a>; a DC motor drives the
axis and is the source of torque. The Harmonic Drive is the gearbox that steps down
the rotational speed while stepping up the torque. There are optical encoders on both
sides of the Harmonic Drive; these encoders will henceforth be referred to as “*motor-side*”
and “*load-side*” encoders. The link is connected to the gearbox’s output shaft via a rigid
coupling. Currently, In the robot software, the motor side encoder is used only to measure
the joint velocity and the load side encoder is used to measure the joint angle. However,
this thesis aims to find another application for these encoders

### 3.1.1. DC Motor

The motor converts electric current  to mechanical torque which drives the robot's joint. According to Taghirad et al. <cite id="swimi"><a href="#zotero%7C17740253%2F7Y2D2ALW">[2]</a></cite>, The relationship between the input current and the output torque of a motor can be expressed as
\begin{equation}
K_m \times I = J_m \ddot\theta + \tau_{f_m} + \tau_{out},\tag{3.1}
\end{equation}
where  $I$ is the motor current, and $K_m$  is the motor-torque-constant, $J$ is the motor inertia, $\tau_{out}$ is the output torque, $\ddot\theta$ is the acceleration of the motor shaft, and $\tau_{f_m}$ is the motor's frictional torque. 

### 3.1.2. Encoders
The motor-side and load-side encoders are both 24-bit optical encoders. These encoders provide both absolute and incremental measurements. The absolute track is used only during startup and afterwards, the encoder uses the incremental measurements. These encoders are used in the feedback loop of the control system for measuring the position and velocity of the joint. If the gearbox is rigid, the readings from both the encoders will be
\begin{equation}
\frac{\theta_m}{N} = \theta_l, \tag{3.2}
\end{equation} 
where $N$ is the gear ratio, $\theta_m$ is the motor-side encoder reading and $\theta_l$ is the load-side encoder reading. This implies that in scenarios where the gearbox is not prone to deformation, only one encoder placed on either the input or the output side of the gearbox is required.

### 3.1.3. Harmonic Drive Transmission

<figure style="text-align: center;">
<img src="./Resources/harmonic.jpg" style="width:50%">
<figcaption align = "center"> <b>Figure 3.2.: </b>Elements of Harmonic Drive Transmission. Source: <cite id="n2ibt"><a href="#zotero%7C17740253%2FNU8U5VH9">[3]</a></cite>. </figcaption>
</figure>


The Harmonic Drive Transmission (HDT) comprises three main parts: a wave generator,
flexible spline or flexspline and circular spline. The wave generator is the input of the
harmonic drive transmission and has an elliptical shape; when rotated, causes the flexible spline to follow its elliptical shape and to engage with the circular spline. Although
flexible, the flex spline is torsionally stiff enough to transmit large torques. According
to Taghirad et al. <cite id="106a8"><a href="#zotero%7C17740253%2F7Y2D2ALW">[2]</a></cite>, HDT’s advantages include "high torque capacity, concentric
geometry, lightweight and compact design, zero backlash, high efficiency, and back driveability". But they suffer from "high flexibility, resonance vibration, friction and structural
damping nonlinearities". 

#### **Stiffness of Harmonic Drive Transmission**

When transmitting torque, the flexspline being flexible undergoes torsional deformation. A plot of the torsion angle against the torque is show in <a href="#fig3.3a">Figure 3.3 (a)</a>. An approximation of this torque-torsional relationship of the HDT is usually provided by the manufacturer in the datasheet in the form of a piecewise linear function as shown in <a href="#fig3.3b">Figure 3.3 (b)</a>.

<figure style="text-align: center;">
    <div style="display: inline-block;">
        <img src="./Resources/torque-torsion-angle-diagram.png" id=fig3.3a style="width:100%;">
        <figcaption>(a) Torsion vs Torque </figcaption>
    </div>
    <div style="display: inline-block; margin-left: 20px;">
        <img src="./Resources/spring-constant-diagram.png" id=fig3.3b style="width:100%;">
        <figcaption>(b) Piecewise linear approximation of Torsion vs Torque</figcaption>
    </div>
    <figcaption style="margin-top: 10px;"><b>Figure 3.3.: </b>Plots from datasheet of SHG-SHF_Gear Units</figcaption>
</figure>

HDTs do not follow Equation 3.2 due to this behaviour and hence, the two separate encoders are generally used in such systems involving HDTs. The relation between motor-side and load-side encoder readings in such systems is given by 

\begin{equation}
\frac{\theta_m}{N } = \theta_l - \theta_e, \tag{3.3}
\end{equation}
where $ \theta_e$ is the torsion angle. Below is an algorithmic implementation of this torsion-torque relationship.

In [3]:
def torsion_to_torque(encoder_error, T1, T2, T3, K1, K2, K3):
    # encoder error in counts
    # torque in Nm
    count_to_radian = 2 * np.pi / (2**24)
    torsion = torsion * count_to_radian
    THETA1 = T1 / K1
    THETA2 = THETA1 + (T2 - T1) / K2
    return (torsion * K1 if torsion < THETA1 else
            T1 + (torsion - THETA1) * K2 if torsion < THETA2 else
            T2 + (torsion - THETA2) * K3)

## 3.2. Kinematics

The kinematics of a robot used in the thesis are introduced in this section along with
some basic concepts and algorithmic implementations. The first subsection discusses
the conversions between coordinate frames and the selected convention. Next, a brief
explanation of the Geometric Jacobian computation procedure and forward kinematics is
given

### 3.2.1. Denavit-Hartenberg Convention and Transformation Matrix

Although reference frames attached to the links can be chosen arbitrarily, it is practical to select them systematically. A widely used convention for describing the 
kinematic geometry of robotic arms is the Denavit-Hartenberg (DH) convention <cite id="e49tg"><a href="#zotero%7C17740253%2F3J5DS9LQ">[4]</a></cite> and is followed in this thesis.

The homogeneous transformation between frame $i$ and $i-1$ is obtained by multiplying the rotation $\mathbf{T}_{R_z}$ around the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_z}$ along the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_x}$ along the $\hat{\mathbf{x}}_i$-axis, and the rotation $\mathbf{T}_{R_x}$ around the $\hat{\mathbf{x}}_i$-axis, as follows:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = \mathbf{T}_{R_z}(\theta_i) \mathbf{T}_{T_z}(d_i) \mathbf{T}_{T_x}(a_i) \mathbf{T}_{R_x}(\alpha_i)\tag{3.4}
\end{equation}

Combining these, the homogeneous transformation matrix is:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = 
\begin{bmatrix}
c_{\theta_i} & -s_{\theta_i} c_{\alpha_i} & s_{\theta_i} s_{\alpha_i} & a_i c_{\theta_i} \\
s_{\theta_i} & c_{\theta_i} c_{\alpha_i} & -c_{\theta_i} s_{\alpha_i} & a_i s_{\theta_i} \\
0 & s_{\alpha_i} & c_{\alpha_i} & d_i \\
0 & 0 & 0 & 1 \tag{3.5}
\end{bmatrix}
\end{equation}

In this expression, $c_{\theta_i}$ and $s_{\theta_i}$ represent $\cos(\theta_i)$ and $\sin(\theta_i)$, respectively. Notably, the homogeneous transformation ${}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i)$ depends on a single variable once the reference frame is established and the other parameters are fixed. This variable is determined by the type of joint: $d_i$ for prismatic joints and $\theta_i$ for revolute joints. A more elaborate theory can be found in <cite id="zhen5"><a href="#zotero%7C17740253%2FLY2FP5ZV">[5]</a></cite>.

In [1]:
def dh_transform(a, alpha, d, theta):
    """
    Compute Denavit-Hartenberg transformation matrix.
    """
    return np.array([
        [np.cos(theta), -np.sin(theta) * np.cos(alpha),  np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
        [np.sin(theta),  np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
        [0,             np.sin(alpha),                 np.cos(alpha),                 d],
        [0,             0,                             0,                             1]
    ])

### 3.2.2. Forward Kinematics

In a robotic manipulator, the links are generally rigid bodies that form the robot’s structure, while the joints are the movable components that join two or multiple links. Joints can be with an actuator (active joints), or without an actuator (passive joints). Joints
commonly found in serial robots are usually prismatic or revolute. Each type of such
joint adds one degree of freedom (DOF) to the system: the joint position di for prismatic
joints or the joint angle qi for revolute joints. The entire kinematic structure is typically
referred to as a chain, which can be of two types:
* Open chain: In this configuration, the two extremes, also known as the base and the
end-effector are connected by a single path in the structure.
*  Closed chain: The kinematic structure of this configuration has loops.

The robot used in this thesis consists only of active revolute joint in an open chain
configuration.
A common problem in robotics is determining the Cartesian position and orientation
$x ∈ R^6$ of a specific point on a link given the joint angles $q ∈ R^n$. This problem is known
as forward kinematics, which can be efficiently solved using homogeneous transformations.
The position of the origin of the end-effector’s reference frame in the base frame can be
determined using the following equation:

\begin{equation}
{}^b\mathbf{T}_e(\mathbf{q}) = {}^b\mathbf{T}_0 \, {}^0\mathbf{T}_1(q_1) \cdots {}^{n-1}\mathbf{T}_e(q_n) \, {}^n\mathbf{T}_e,\tag{3.6}
\end{equation}

where the two constant homogeneous transformations required to convert a point from frame 0 to the base frame and from the end-effector frame to frame $n$, respectively, are denoted by ${}^b\mathbf{T}_0$ and ${}^n\mathbf{T}_e$.

 The following algorithmic implementation computes the transformation between all reference frames in the robot defined in the DH parameters.

In [3]:
def compute_forward_kinematics(joint_angles, dh_params):
    """
    Compute the forward kinematics for a robot given DH parameters and joint angles.

    :param dh_params: List of DH parameters [(a, alpha, d, theta), ...]
    :param joint_angles: List of joint angles [theta1, theta2, ..., thetaN]
    :return: List of transformation matrices from base to each joint and the final transformation matrix T_0_n
    """
    num_joints = len(joint_angles)
    T = np.eye(4)
    transformations = [T.copy()]
    
    # Compute forward kinematics to get transformations
    for i, (a, alpha, d, theta) in enumerate(dh_params):
        theta += joint_angles[i]  # Update theta with joint angle
        T_i = dh_transform(a, alpha, d, theta)
        T = np.dot(T, T_i)
        transformations.append(T.copy())
    
    T_0_n = transformations[-1]  # Final transformation matrix to the TCP (end-effector)
    return transformations, T_0_n

### 3.2.3. Geometric Jacobian

The geometric Jacobian matrix, $J(q) ∈ R^{(6×n)}$, transforms the joint space velocities $q̇$, into Cartesian space velocities $ẋ$. This relationship is represented by the equation:
\begin{equation}
\dot{\mathbf{x}} = 
\begin{bmatrix}
\dot{\mathbf{p}} \\
\boldsymbol{\omega}
\end{bmatrix} \tag{3.7}
= \mathbf{J}(\mathbf{q}) \dot{\mathbf{q}},
\end{equation}
where $\dot p = [ \dot x, \dot y, \dot z ]^T $represents the linear velocities and $\omega = [ \omega_x, \omega_y, \omega_z ]^T$ represents the angular velocities of the robot's end-effector in the base frame. The geometric Jacobian is calculated based on the robot's geometry at a specific instance. An elaborate theoretical background can be found in <cite id="63bkr"><a href="#zotero%7C17740253%2FLY2FP5ZV">[5]</a></cite>, an algorithmic implementation is shown below. 

In [4]:
def compute_geometric_jacobian(transformations):
    """
    Compute the geometric Jacobian for a robot given the transformation matrices.

    :param transformations: List of transformation matrices from the base to each joint
    :return: Geometric Jacobian matrix (6 x N) in the base frame
    """
    num_joints = len(transformations) - 1  # Excluding the base frame
    jacobian_base = np.zeros((6, num_joints))
    
    # End-effector position
    T_0_n = transformations[-1]
    o_n = T_0_n[:3, 3]

    # Compute each column of the Jacobian in the base frame
    for i in range(num_joints):
        T_0_i = transformations[i]
        o_i = T_0_i[:3, 3]
        z_i = T_0_i[:3, 2]

        jacobian_base[:3, i] = np.cross(z_i, (o_n - o_i))  # Linear part
        jacobian_base[3:, i] = z_i  # Angular part

    return jacobian_base

## 3.3. LARA 8 Kinematic Description


<figure>
  <img src="Resources/Lara8.png" alt="Lara8" title="Lara8" id=Lara8 style="max-width: 100%;">
  <figcaption><b>Figure 3.4.:</b> Frames and DH-convention for LARA8. The image is a property of NEURA
Robotics GmbH.</figcaption>
</figure>



The robot used for the experiments in this thesis is LARA 8, a lightweight collaborative robotic
arm from the company NEURA Robotics GmbH. It is a 6-axis serial manipulator
that has six independently actuated revolute joints and seven links. The maximum payload is 8kg. <a href="#Lara8">Figure 3.4</a> shows the DH
frames, the DH parameters and the actuated joints of LARA 8. The DH-parameters
are given in <a href="#table3.1">Table 3.1</a>
<a id="table3.1"></a> 
<table style="width: 50%; margin: auto; border-collapse: collapse;">
  <caption style="caption-side: bottom; text-align: center;"><b>Table 3.1.:</b> DH Parameters of LARA8</caption>
  <thead>
    <tr>
      <th>Frame</th>
      <th>&theta; (rad)</th>
      <th>d (m)</th>
      <th>a (m)</th>
      <th>&alpha; (rad)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>K1</td>
      <td>0</td>
      <td>0.26</td>
      <td>0</td>
      <td>\(\frac{\pi}{2}\)</td>
    </tr>
    <tr>
      <td>K2</td>
      <td>\(\frac{\pi}{2}\)</td>
      <td>0</td>
      <td>0.6</td>
      <td>\(\pi\)</td>
    </tr>
    <tr>
      <td>K3</td>
      <td>\(\frac{\pi}{2}\)</td>
      <td>0</td>
      <td>0</td>
      <td>\(\frac{\pi}{2}\)</td>
    </tr>
    <tr>
      <td>K4</td>
      <td>0</td>
      <td>0.7</td>
      <td>0</td>
      <td>\(-\frac{\pi}{2}\)</td>
    </tr>
    <tr>
      <td>K5</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>\(\frac{\pi}{2}\)</td>
    </tr>
    <tr>
      <td>K6</td>
      <td>0</td>
      <td>0.1735</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>



## 3.4. Dynamics

This section lays out the theoretical groundwork for understanding the dynamics of serial manipulators. First, equations of motion for a generic open-chain robotic manipulator is presented. These differential equations form the dynamic model of the robot, describing the relationship between joint positions ($q$), velocities ($\dot{q}$), and accelerations ($\ddot{q}$) with the torques ($\tau$) applied to these joints by actuators.

Following this, the topic of friction, its modelling and algorithmic implementation is introduced. The final part of the section addresses the calculation and algorithmic implementation necessary for converting torques at the Tool Center Point (TCP) to joint torques and vice versa.

### 3.4.1. Equations of motion
The general equation of motion for the robot under study, which is an open-chain manipulator is given below. This equation presents the joint torque $\tau$ required to move the robot's joints in accordance with a desired trajectory with joint positions $q$, joint velocities $\dot{q}$, and joint accelerations $\ddot{q}$. 
\begin{equation}
 \underbrace{M(\mathbf{q}) \ddot{\mathbf{q}} + \mathbf{C}(\mathbf{q}, \dot{\mathbf{q}}) + \mathbf{g}(\mathbf{q}) + \mathbf{\tau_f}}_{\text{Torque Feedforward}}+ \mathbf{\tau_{ext}}= \mathbf{\tau}.
 \tag{3.8}
\end{equation}
Here, $M (q) ∈ R^{n×n}$ is a symmetric and positive definite matrix representing the robot’s inertia, $C(q, q̇) ∈ R^n$ is the vector containing the Coriolis, centrifugal and gyroscopic terms and $g(q) ∈ R^n$ is the gravity vector, $\mathbf{\tau_f}$ is the frictional torque and $\mathbf{\tau_{ext}}$ is the torque in the joint due to interaction of the robot with the environment. The term labelled as 'Torque Feedforward' in the equation can be computed for the desired trajectory, providing a preliminary estimate of the joint torques. This feedforward torque is fed to the PID controllers in each joint to achieve the desired joint position, velocity, and acceleration. Although the controller will adjust the torque applied to the joint as needed, the feedforward torque serves as an initial estimate. The detailed derivation and more discussions can be found in <cite id="us6nv"><a href="#zotero%7C17740253%2FLY2FP5ZV">[5]</a></cite>.

### 3.4.2. Friction Model

Friction occurs in most mechanical systems where there is relative motion between two mechanical components. In order to provide a good torque estimate following Equation 3.8, it is essential to have a good model of the joint's friction. However, friction is very complex to model accurately and is affected by many factors such as joint velocity, temperature, load, etc. To model the frictional torque, many simplifications have to be done. According to <cite id="mnqix"><a href="#zotero%7C17740253%2FYSG8J3F3">[6]</a></cite> friction dynamics can be simplified into the following four components.

#### **Static Friction**

Static friction is the component of friction which is present between two surfaces in contact when there is no relative motion and an external force is applied. Static friction acts in the direction opposite to the applied force that is attempting to initiate motion. Its magnitude is equal to the external force ($f_e$) until the external force exceeds the maximum static frictional force ($f_s$), beyond which static friction is overcome, and motion begins.
\begin{equation}
[
\tau_f =
\begin{cases} 
f_e, & |f_e| < f_s \\
f_s \delta(\dot{x}) \text{sgn}(f_e), & |f_e| \geq f_s 
\end{cases}
] \tag{3.9}
\end{equation}
where
\begin{equation}
[
\delta(\dot{x}) = \begin{cases} 
1, & \dot{x} = 0 \\
0, & \dot{x} \neq 0 
\end{cases}
\quad , \quad
\text{sgn}(f_e) = \begin{cases} 
+1, & f_e > 0 \\
-1, & f_e < 0 
\end{cases}
]
 \tag{3.10}
\end{equation}

#### **Coulomb friction**
Coulomb friction opposes the motion and is proportional to the normal contact force ($f_n$). 

\begin{equation}
\tau_f =  \mu f_n \text{sgn}(\dot{x}),
 \tag{3.11}
\end{equation}
where $\mu$ is a constant called friction coefficient.

#### **Viscous friction**
This is a velocity-dependent component of friction and is proportional to velocity. It is given by the equation,

\begin{equation}
\tau_f = f_v \dot(x),
 \tag{3.12}
\end{equation}
where $f_v$ is a constant called a viscous coefficient.

#### **Friction model**

Although multiple friction models exists <cite id="sxyi9"><a href="#zotero%7C17740253%2FYSG8J3F3">[6]</a></cite> which cover all these components, however the friction model used in this thesis is from <cite id="3f4kj"><a href="#zotero%7C17740253%2FXA8NYAMV">[7]</a></cite> and is given by 

\begin{equation}
f(v) = a_1 (\tanh(b_1 v) - \tanh(b_2 v)) + a_2 \tanh(b_3 v) + a_3 v,
 \tag{3.13}
\end{equation}

where $a_1,a_2,a_3,b_1,b_2,b_3$ are all positive paramters which are found emperically. 

### 3.4.4. Conversion of Wrench at TCP to Joint torques


The conversion of external forces and torques applied to the end-effector of a 6-DOF robot to joint torques is achieved through the Jacobian matrix $\mathbf{J}(\mathbf{q})$, which relates joint velocities to the end-effector linear and angular velocities.

#### **External Force and Torque Representation**

External forces $\mathbf{F}_{\text{ext}}$ and torques $\mathbf{\tau}_{\text{ext}}$ are combined into a wrench vector $\mathbf{W}_{\text{ext}}$:

$$
\mathbf{W}_{\text{ext}} = \begin{bmatrix} \mathbf{F}_{\text{ext}} \\ \mathbf{\tau}_{\text{ext}} \end{bmatrix}
$$


#### **Conversion to Joint Torques**

The joint torques $\mathbf{\tau}_{\text{joint}}$ are computed by:

\begin{equation}
\mathbf{\tau}_{\text{joint}} = \mathbf{J}^\top(\mathbf{q}) \mathbf{W}_{\text{ext}}. \tag{3.14}
\end{equation}

This equation projects the external wrench into the joint space, providing the necessary torques required at each joint to counteract the external forces and torques.


Note that the Jacobian and the wrench are defined in the same coordinate frame. In this thesis, wrench $W_{ext}$ is always defined in the TCP frame and hence the Jacobian is also calculated in the TCP frame. An algorithmic implementation is given below.

In [6]:
def convert_jacobian_to_tcp(jacobian_base, T_0_n):
    """
    Convert a Jacobian from the base frame to the TCP frame.
    
    :param jacobian_base: Jacobian matrix in the base frame (6 x 6)
    :param T_0_n: Transformation matrix from the base to the TCP (4 x 4)
    :return: Jacobian matrix (6 x 6) in the TCP frame
    """
    R_0_n = T_0_n[:3, :3]
    T_base_to_tcp = np.block([
        [R_0_n.T, np.zeros((3, 3))],
        [np.zeros((3, 3)), R_0_n.T]
    ])
    
    jacobian_tcp = np.dot(T_base_to_tcp, jacobian_base)
    return jacobian_tcp

 ## 3.5. Machine Learning
This section briefly discusses some machine learning fundamentals used in this thesis like
the neural network, different types of recurrent units such as RNN, GRU and LSTM

### 3.5.1. Neural Network
A neural network is a computational model that is based on the information-processing
mechanisms similar to biological neural networks found in the human brain. It is made up
of one or more layers of interconnected nodes, or "neurons". Each connection is a weighted
pathway that modifies as learning advances. Through a process called backpropagation <cite id="bwq6k"><a href="#zotero%7C17740253%2FNKAIXGX3">[8]</a></cite>, neural networks modify their weights according to the loss computed during
each training step, neural networks can identify patterns and relationships in data. Neural networks can learn complex non-linear relationships in data which analytical
methods often struggle to do.

A mathematical model of a single neuron with multiple inputs $x1, . . . , xN$ and an output
$y$, and a series of learnable weights $w1, . . . , wN$ is given in Equation 3.15
\begin{equation}
y = f\left( \sum_{i=1}^{N} x_i w_i \right) + b,\tag{3.15}
\end{equation}

where $f$ is an activation function like tanh, sigmoid, etc., and b is a learnable parameter
called bias.

### 3.5.2 Recurrent Neural Networks
Recurrent Neural Network (RNN) is a kind of neural network that is specifically made for
sequential data like time series data, natural language, or video frames, where the input
is a sequence. RNNs contain loops in their architecture which help them to retain information
over time steps, unlike standard neural networks. Because of this looping process, RNNs
can retain a recollection of past inputs, which makes them ideal for jobs involving temporal dynamics or contextual awareness. RNNs are useful for applications like language
modelling, speech recognition, and sequence prediction because they can learn dependencies over time by processing inputs sequentially. RNNs face problems with vanishing and
exploding gradients, which can make training difficult, particularly for lengthy sequences
<cite id="xlro6"><a href="#zotero%7C17740253%2F7JPSKM68">[9]</a></cite>.


An elaborate theory on RNN can be found in <cite id="4tn8m"><a href="#zotero%7C17740253%2F7JPSKM68">[9]</a></cite>.

### 3.5.3 Long Short Term Memory
Long Short Term Memory (LSTM) <cite id="sbf2o"><a href="#zotero%7C17740253%2FFQBVI9RB">[10]</a></cite> is a type of Recurrent Neural Network which
was developed to address the issue of vanishing and exploding gradients in RNNs. In
this architecture three gates, namely the input gate, forget gate, and output gate were
introduced to control the information flow, allowing it to selectively remember, update
and forget information over long periods. This makes LSTM particularly useful in cases
where a long temporal relation between inputs needs to be maintained by the network such
as natural language processing.

### 3.5.4 Gated Recurrent Unit
Gated Recurrent Unit (GRU) <cite id="zmp3j"><a href="#zotero%7C17740253%2FYJ447BA4">[11]</a></cite> was introduced as a simpler and a more faster
alternative to LSTM. It uses a similar mechanism of gates to remember, update and
forget information, however, unlike LSTM it uses only two gates for this purpose, namely
the update gate and reset gate. This leads to fewer parameters and simple architecture
resulting in faster training and inference times. GRUs are preferred over LSTMs in tasks
where the training data is limited, computational power is low, or quick iterations are
necessary, such as real-time processing or applications with long sequences and fewer
computational requirements.

## References

<!-- BIBLIOGRAPHY START -->
<div class="csl-bib-body">
  <div class="csl-entry"><i id="zotero|17740253/4JU53NME"></i>
    <div class="csl-left-margin">[1]</div><div class="csl-right-inline">H. Zhang, S. Ahmad, and G. Liu, “Torque Estimation for Robotic Joint With Harmonic Drive Transmission Based on Position Measurements,” <i>IEEE Trans. Robot.</i>, vol. 31, no. 2, pp. 322–330, 2015, doi: <a href="https://doi.org/10.1109/TRO.2015.2402511">10.1109/TRO.2015.2402511</a>.</div>
  </div>
  <div class="csl-entry"><i id="zotero|17740253/7Y2D2ALW"></i>
    <div class="csl-left-margin">[2]</div><div class="csl-right-inline">H. D. Taghirad and P. R. Belanger, “An experimental study on modelling and identification of harmonic drive systems,” in <i>Proceedings of 35th IEEE Conference on Decision and Control</i>, Kobe, Japan, 1996, vol. 4, pp. 4725–4730. doi: <a href="https://doi.org/10.1109/CDC.1996.577625">10.1109/CDC.1996.577625</a>.</div>
  </div>
  <div class="csl-entry"><i id="zotero|17740253/NU8U5VH9"></i>
    <div class="csl-left-margin">[3]</div><div class="csl-right-inline">M. Majchrák, R. Kohar, M. Lukac, and R. Skyba, “Analysis of harmonic gearbox tooth contact pressure,” <i>IOP Conference Series: Materials Science and Engineering</i>, vol. 659, p. 012068, Oct. 2019, doi: <a href="https://doi.org/10.1088/1757-899X/659/1/012068">10.1088/1757-899X/659/1/012068</a>.</div>
  </div>
  <div class="csl-entry"><i id="zotero|17740253/3J5DS9LQ"></i>
    <div class="csl-left-margin">[4]</div><div class="csl-right-inline">J. Denavit and R. S. Hartenberg, “A Kinematic Notation for Lower-Pair Mechanisms Based on Matrices,” <i>Journal of Applied Mechanics</i>, vol. 22, no. 2, pp. 215–221, Jun. 2021, doi: <a href="https://doi.org/10.1115/1.4011045">10.1115/1.4011045</a>.</div>
  </div>
  <div class="csl-entry"><i id="zotero|17740253/LY2FP5ZV"></i>
    <div class="csl-left-margin">[5]</div><div class="csl-right-inline">A. Ghosal, <i>ROBOTICS: FUNDAMENTAL CONCEPTS AND ANALYSIS</i>. OUP India, 2006. [Online]. Available: <a href="https://books.google.de/books?id=GsRhAAAACAAJ">https://books.google.de/books?id=GsRhAAAACAAJ</a></div>
  </div>
  <div class="csl-entry"><i id="zotero|17740253/YSG8J3F3"></i>
    <div class="csl-left-margin">[6]</div><div class="csl-right-inline">J. Na, Q. Chen, and X. Ren, “Friction Dynamics and Modeling,” in <i>Adaptive Identification and Control of Uncertain Systems with Non-smooth Dynamics</i>, Elsevier, 2018, pp. 11–18. doi: <a href="https://doi.org/10.1016/B978-0-12-813683-6.00003-9">10.1016/B978-0-12-813683-6.00003-9</a>.</div>
  </div>
  <div class="csl-entry"><i id="zotero|17740253/XA8NYAMV"></i>
    <div class="csl-left-margin">[7]</div><div class="csl-right-inline">C. Makkar, W. E. Dixon, W. G. Sawyer, and G. Hu, “A new continuously differentiable friction model for control systems design,” in <i>Proceedings, 2005 IEEE/ASME International Conference on Advanced Intelligent Mechatronics.</i>, Monterey, CA, 2005, pp. 600–605. doi: <a href="https://doi.org/10.1109/AIM.2005.1511048">10.1109/AIM.2005.1511048</a>.</div>
  </div>
  <div class="csl-entry"><i id="zotero|17740253/NKAIXGX3"></i>
    <div class="csl-left-margin">[8]</div><div class="csl-right-inline">“Learning representations by back-propagating errors | Nature.” Accessed: Aug. 25, 2024. [Online]. Available: <a href="https://www.nature.com/articles/323533a0">https://www.nature.com/articles/323533a0</a></div>
  </div>
  <div class="csl-entry"><i id="zotero|17740253/7JPSKM68"></i>
    <div class="csl-left-margin">[9]</div><div class="csl-right-inline">R. M. Schmidt, “Recurrent Neural Networks (RNNs): A gentle Introduction and Overview.” arXiv, Nov. 23, 2019. doi: <a href="https://doi.org/10.48550/arXiv.1912.05911">10.48550/arXiv.1912.05911</a>.</div>
  </div>
  <div class="csl-entry"><i id="zotero|17740253/FQBVI9RB"></i>
    <div class="csl-left-margin">[10]</div><div class="csl-right-inline">S. Hochreiter and J. Schmidhuber, “Long Short-Term Memory,” <i>Neural Computation</i>, vol. 9, no. 8, pp. 1735–1780, Nov. 1997, doi: <a href="https://doi.org/10.1162/neco.1997.9.8.1735">10.1162/neco.1997.9.8.1735</a>.</div>
  </div>
  <div class="csl-entry"><i id="zotero|17740253/YJ447BA4"></i>
    <div class="csl-left-margin">[11]</div><div class="csl-right-inline">D. Bahdanau, K. Cho, and Y. Bengio, “Neural Machine Translation by Jointly Learning to Align and Translate,” 2014, doi: <a href="https://doi.org/10.48550/ARXIV.1409.0473">10.48550/ARXIV.1409.0473</a>.</div>
  </div>
</div>
<!-- BIBLIOGRAPHY END -->